#### Core 2-04: Instability Diagnostics

목적:
- 예측값을 설계 트리거로 사용했을 때 발생하는
  **의사결정 불안정성(decision instability)** 을 수치로 고정한다.

중요 원칙:
- 성능 평가 ❌
- cutoff ❌
- 예측 정확도 ❌
- 오직 decision dynamics만 분석한다.

이 노트북은 Core 2의 관찰 결과를
"느낌"이 아니라 "지표"로 고정하는 단계다.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# Core 2 산출물 로딩
artifact_dir = Path("../artifact/core2")

prediction_trace = pd.read_csv(
    artifact_dir / "prediction_trace.csv"
)

prediction_trace.head()

,antibody_key,step,sequence_current,pred_score,pred_score_delta,decision,mutation_id_applied,intervention_count_cum
0,GDPa1-001,1,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,2.013943,0.013943,MUTATE,GDPa1-001_mut0_1,1
1,GDPa1-001,2,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,1.966444,-0.047499,HOLD,NaN,1
2,GDPa1-001,3,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,1.943947,-0.022497,HOLD,NaN,1
3,GDPa1-001,4,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,1.916268,-0.027679,HOLD,NaN,1
4,GDPa1-001,5,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,1.939915,0.023647,MUTATE,GDPa1-001_mut1_5,2


In [2]:
prediction_trace.info()
# 항체별 step 범위 확인
prediction_trace.groupby("antibody_key")["step"].max()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   antibody_key            60 non-null     object 
 1   step                    60 non-null     int64  
 2   sequence_current        60 non-null     object 
 3   pred_score              60 non-null     float64
 4   pred_score_delta        60 non-null     float64
 5   decision                60 non-null     object 
 6   mutation_id_applied     29 non-null     object 
 7   intervention_count_cum  60 non-null     int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 3.9+ KB


antibody_key
GDPa1-001    20
GDPa1-045    20
GDPa1-183    20
Name: step, dtype: int64

 toggle_rate 계산
(MUTATE ↔ HOLD 전환 빈도)
정의 (고정)
toggle = 직전 step과 decision이 달라지는 경우

In [ ]:
def compute_toggle_rate(df):
    decisions = df["decision"].values
    toggles = 0
    
    for i in range(1, len(decisions)):
        if decisions[i] != decisions[i - 1]:
            toggles += 1
    
    return toggles / (len(decisions) - 1)

toggle_results = []

for ab, group in prediction_trace.groupby("antibody_key"):
    rate = compute_toggle_rate(group)
    toggle_results.append({
        "antibody_key": ab,
        "toggle_rate": rate
    })

toggle_df = pd.DataFrame(toggle_results)
toggle_df # 	•	값이 높을수록 → 예측 변화에 설계 판단이 과민하게 반응

,antibody_key,toggle_rate
0,GDPa1-001,0.473684
1,GDPa1-045,0.315789
2,GDPa1-183,0.368421


intervention_burst 계산
(연속 MUTATE 구간 길이)
정의 (고정)
MUTATE가 연속으로 이어진 구간의 길이

In [ ]:
def compute_intervention_bursts(df):
    bursts = []
    current_burst = 0
    
    for d in df["decision"]:
        if d == "MUTATE":
            current_burst += 1
        else:
            if current_burst > 0:
                bursts.append(current_burst)
            current_burst = 0
    
    if current_burst > 0:
        bursts.append(current_burst)
    
    return bursts

burst_results = []

for ab, group in prediction_trace.groupby("antibody_key"):
    bursts = compute_intervention_bursts(group)
    
    burst_results.append({
        "antibody_key": ab,
        "num_bursts": len(bursts),
        "mean_burst_length": np.mean(bursts) if bursts else 0,
        "max_burst_length": np.max(bursts) if bursts else 0
    })

burst_df = pd.DataFrame(burst_results)
burst_df # burst가 존재한다 = 개입이 “단발”이 아니라 “연쇄”로 이어짐 / 예측 트리거가 설계를 멈추지 못함

,antibody_key,num_bursts,mean_burst_length,max_burst_length
0,GDPa1-001,5,1.8,3
1,GDPa1-045,4,3.5,6
2,GDPa1-183,4,1.5,3


unnecessary_intervention_proxy 계산

개념 정의 (Core 2용, 단순·고정)

MUTATE가 발생했지만
바로 다음 step에서 pred_score가 회복(상승) 되는 경우
→ “굳이 안 해도 됐을 개입”의 proxy

❗ 생물학적 정당성 아님
❗ 구조적 과잉 반응 측정용 지표

In [6]:
def compute_unnecessary_intervention(df):
    count = 0
    
    for i in range(len(df) - 1):
        if df.iloc[i]["decision"] == "MUTATE":
            # 다음 step에서 score가 회복되면 unnecessary로 간주
            if df.iloc[i + 1]["pred_score_delta"] > 0:
                count += 1
    
    return count

unnecessary_results = []

for ab, group in prediction_trace.groupby("antibody_key"):
    cnt = compute_unnecessary_intervention(group)
    
    unnecessary_results.append({
        "antibody_key": ab,
        "unnecessary_intervention_count": cnt,
        "total_mutations": (group["decision"] == "MUTATE").sum()
    })

unnecessary_df = pd.DataFrame(unnecessary_results)
unnecessary_df

,antibody_key,unnecessary_intervention_count,total_mutations
0,GDPa1-001,4,9
1,GDPa1-045,10,14
2,GDPa1-183,2,6


In [7]:
diagnostics_df = (
    toggle_df
    .merge(burst_df, on="antibody_key")
    .merge(unnecessary_df, on="antibody_key")
)

diagnostics_df # 진단 지표 통합 테이블 생성

output_path = artifact_dir / "core2_instability_diagnostics.csv"
diagnostics_df.to_csv(output_path, index=False)

output_path # Core 2 불안정성 요약 저장 

PosixPath('../artifact/core2/core2_instability_diagnostics.csv')

In [8]:
import json

policy_path = artifact_dir / "policy_summary.json"

with open(policy_path, "r") as f:
    policy_data = json.load(f)

policy_data["instability_diagnostics"] = diagnostics_df.to_dict(orient="records")

with open(policy_path, "w") as f:
    json.dump(policy_data, f, indent=2)

policy_path # policy_sumary.json에 결과 append (정책+결과를 한 파일에 같이 보관하고 싶을 때 사용)

PosixPath('../artifact/core2/policy_summary.json')

## Core 2 고정 결론 (개발자용)

- 예측값을 설계 트리거로 사용할 경우,
  설계 개입은 단발로 끝나지 않고 반복·누적된다.
- 예측 변동 구간에서는 MUTATE/HOLD 판단이 빈번히 교차한다.
- 일부 개입은 곧 회복되는 구간에서도 발생하며,
  이는 구조적으로 불필요한 개입을 유발한다.